In [1]:

import os

text_paths = list(map( lambda x: './data/honglou/'+x,os.listdir('./data/honglou/')))

# text_paths

In [2]:
class File(object):
    def __init__(self, ):
        pass

    def write(self, path,  body, encoding='utf8'):
        with open(path, 'w', encoding=encoding) as f:
            f.write(body)
    def append(self, path, body, encoding='utf8'):
        with open(path, 'a', encoding=encoding) as f:
            f.write(body)
    def read_to_str(self, path, encoding='utf8'):
        with open(path, 'r', encoding=encoding) as f:
            return ''.join(f.readlines()).replace('\n','')
    def file_map(self, path, fn, encoding='utf8'):
        with open(path, 'r', encoding='utf8') as f:
            for sentence in f.readlines():
                fn(sentence, path)
                
file = File()
# file.read_to_str('./data/honglou/第100章.txt')
# cut red_water into documents

# chapter_mark = '------------\n'
# chapter_count = 0
# with open('./data/red_water.txt', encoding='utf8') as fp:
#     chapter_body = ''
#     for sentence in fp.readlines():
#         if sentence == chapter_mark:
#             # skip the first time
#             if chapter_count == 0: 
#                 chapter_count += 1
#                 continue
#             else:
#                 # write the text
#                 chapter_title = f'第{chapter_count}章.txt'
#                 file.write('./data/honglou/'+ chapter_title, chapter_body)
#                 # clean variables
#                 chapter_count += 1
#                 chapter_body = '' 
#         else: 
#             chapter_body += sentence


In [3]:
import jieba
import numpy as np
class IDF(File):
    def __init__(self, text_paths):
        super(File).__init__()
        self.idf_store = {}
        self.idf_store_pre = {}
        self.N = None
        self.train(text_paths)
    def train(self, text_paths):
        '''
        params:
            text_paths: all doucment paths
        return: 
            idf dictory (key: qi, value: score of idf)
        '''
        self.N  = len(text_paths) #number of document 
        def process_line(sentence, path):
            sentence = self.__clean(sentence)
            words = jieba.cut_for_search(sentence)
            for word in words:
                self.__set_idf_store_pre(word, path)
        for text_path in text_paths:
            self.file_map(text_path, process_line)
        for word,docs in self.idf_store_pre.items():
            self.idf_store[word] = len(docs) 

    def __clean(self,string):
        string = string.strip()
        return string
    def __set_idf_store_pre(self, word, text_path):
        if self.idf_store_pre.get(word, False):
            if text_path not in self.idf_store_pre[word]:
                self.idf_store_pre[word].append(text_path)
        else:
            self.idf_store_pre[word] = [text_path]
    def get(self, word):
        '''
        params:
            word : that you to know its idf
        return:
            word's idf
        '''
        n_qi = self.idf_store.get(word, self.N)
        return np.log( (self.N - n_qi + 0.5) / (n_qi + 0.5) )

# text_paths = ['./data/honglou/第100章.txt',  './data/honglou/第101章.txt']

# idf = IDF(text_paths)
# idf.train(text_paths)
# idf.idf_store

# idf.get('贾雨村')
# idf.get('贾雨')


In [4]:
import jieba

class BM25(object):

    def __init__(self, text_paths):
        self.text_paths = text_paths
        self.fre_dict = {}  # word'd appear frequence
        self.dl =  {} # lenght of document
        #hyporparameters
        self.k1 = 2
        self.b = 0.75

    def train(self, ):
        self.idf = IDF(text_paths)
        # record document length (dl)
        for text_path in self.text_paths:
            file.file_map(text_path, self.__process_sentence)
            self.dl[text_path] = len( file.read_to_str(text_path) )
        # documenet average length
        self.dl['avgdl'] = sum(self.dl.values()) / len(self.text_paths)
    
    def __process_sentence(self, sentence, path):
        def set_fre_dict(key, doc_id):
            if self.fre_dict.get(key, False):
                if self.fre_dict[key].get(doc_id, False):
                    self.fre_dict[key][doc_id] += 1
                else:
                    self.fre_dict[key][doc_id] = 1
            else:
                self.fre_dict[key] = {doc_id: 1}
        sentence = sentence.strip()
        words = jieba.cut(sentence)
        for word in words:
            set_fre_dict(word, path)

    def query(self, sentence, n=8):
        '''
        params:
            sentence: the input sentence
            R: score for word and document
        return:
            score for current text
        '''
        word_scores = []
        words = list(jieba.cut(sentence))
        for text_path in self.text_paths:
            record = []
            for word in words:
                # wi: weight of current word, acrually idf
                w = self.idf.get(word)
                r = self.R(word, text_path)
                record.append(w*r)
            word_scores.append({
                'text_paths': text_path,
                'score': sum(record)
            })
        word_scores.sort(key=lambda x: x['score'], reverse=True)
        return word_scores[:n]

    def R(self, qi,doc_id):
        '''
        params:
            qi: the word factor 
            d: documnet
        return: 
            score for word(qi) and document(d) 
        '''
        dl =  self.dl[doc_id]
        avgdl = self.dl['avgdl']
        try:
            f_i = self.fre_dict[qi][doc_id]
        except Exception as e:
            # print(e)
            f_i = 0
        K = self.k1 * ( 1-self.b + self.b * (dl/ avgdl) )
        R = (f_i * self.k1) / (f_i + K)
        # print(dl, avgdl, f_i, K, R)
        return R

# text_paths = ['./data/honglou/第100章.txt',  './data/honglou/第101章.txt', './data/honglou/第101章.txt']
bm25 = BM25(text_paths)
bm25.train()


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\lenovo\AppData\Local\Temp\jieba.cache
Loading model cost 0.434 seconds.
Prefix dict has been built successfully.


In [8]:
# bm25.query('刘姥姥大观园')
# bm25.query('姥姥观园')
bm25.query('黛玉葬花',)
# bm25.query('贾宝玉')
# list(jieba.cut('林黛玉'))
# bm25.R('宝玉',text_paths[0])
# bm25.idf.get('宝玉')
# bm25.query('宝玉')
# text_paths

[{'text_paths': './data/honglou/第27章.txt', 'score': 2.5858814153760137},
 {'text_paths': './data/honglou/第30章.txt', 'score': 2.5038802146392842},
 {'text_paths': './data/honglou/第72章.txt', 'score': 1.9514468391839017},
 {'text_paths': './data/honglou/第31章.txt', 'score': 1.0251785095897015},
 {'text_paths': './data/honglou/第92章.txt', 'score': 1.0087800990678442},
 {'text_paths': './data/honglou/第28章.txt', 'score': 0.8444882659010499},
 {'text_paths': './data/honglou/第101章.txt', 'score': 0.0},
 {'text_paths': './data/honglou/第102章.txt', 'score': 0.0}]